# <span style=color:red>Gestionnaire d'utilisateurs avec Pandas</span>
---
Objectif :
- Créer un système simple de gestion d'utilisateurs en Python, sans interface graphique.
- Utiliser Pandas pour stocker et manipuler les données des utilisateurs dans un DataFrame.
- Implémenter les fonctionnalités CRUD (Create, Read, Update, Delete) pour gérer les utilisateurs.
- Gérer la connexion de l'administrateur avec un mot de passe hashé.
- Permettre aux utilisateurs de voir la liste des autres utilisateurs après s'être connectés.


In [89]:
import os
import time

import random
import numpy as np
import pandas as pd
import hashlib
from IPython.display import clear_output
from matplotlib import pyplot as plt

Stockage des données :
- Utiliser un DataFrame Pandas pour stocker les informations des utilisateurs (nom d'utilisateur, mot de passe hashé, email).
- Initialiser le DataFrame avec un utilisateur administrateur et un mot de passe hashé (faire un fichier à part pour cette opération).


In [90]:
def hash_password(password):
    """Hacher le mot de pass avec SHA-256"""
    return hashlib.sha256(password.encode()).hexdigest()


In [ ]:
os.makedirs("config", exist_ok=True)
DATA_DIR = "config"

ADMIN_FILE = os.path.join(DATA_DIR, "admin.csv")
USERS_FILE = os.path.join(DATA_DIR, "users.csv")


In [ ]:
def creation_config_une_fois():
    
    # Initialiser le DataFrame avec un utilisateur administrateur
    admin_data = {
        "username": ["admin"],
        "password_hash": [hash_password("pas")],
        "email": ["admin.example@gmail.com"],
        "donne_s":[""]
    }

    admin_df = pd.DataFrame(admin_data)
    users_df = pd.DataFrame(columns=admin_df.columns)

    # Stocke CSV 
    admin_df.to_csv(ADMIN_FILE, index=False)
    users_df.to_csv(USERS_FILE, index=False)



### <span style=color:red>Connexion de l'administrateur :</span>
- Demander à l'utilisateur de saisir son nom d'utilisateur et son mot de passe.
- Vérifier si le nom d'utilisateur existe dans le DataFrame.
- Si oui, comparez le mot de passe saisi avec le mot de passe hashé stocké dans le DataFrame en utilisant une bibliothèque de hachage (par exemple, bcrypt ou hashlib).
- Si les informations de connexion sont correctes, accorder l'accès administrateur.
- Une fois connecté une série de question est posé à l’administrateur en fonction des cas ci dessous


In [93]:
def connection_admin():
    while True:
        try:
            nom = str(input("Entrez le nom d'utilisateur"))
            
            user_row = admin_df[admin_df["username"] == nom]

            if not user_row.empty:
                mot_de_pass = str(input("Entrez le mot de passe"))
                
                if hash_password(mot_de_pass) == user_row.loc[0, "password_hash"]:
                    print("l'accès administrateur est accordé")
                    menu_admin()
                    break
                else:
                    print("Mot de passe incorrect")
                    break
            else:
                print("Nom d'utilisateur introuvable")
                break
        
        except Exception as e:
            print("Les données ne sont pas validées:", e)
            break

In [106]:
def connection_user():
    try:
        users_df = pd.read_csv(USERS_FILE)
        nom = str(input("Entrez le nom d'utilisateur"))
        
        user_row = users_df[users_df["username"] == nom]

        if not user_row.empty:
            mot_de_pass = str(input("Entrez le mot de passe"))
            
            if hash_password(mot_de_pass) == user_row.iloc[0]["password_hash"]:
                print("l'accès a votre compte est accordé")
                menu_user(nom)

            else:
                print("Mot de passe incorrect")

        else:
            print("Nom d'utilisateur introuvable")

    
    except Exception as e:
        print("Les données ne sont pas validées:", e)

        

### <span style=color:red>Création d'utilisateurs (CRUD - Create) :</span>
- Demander à l'administrateur de saisir les informations du nouvel utilisateur
  - nom d'utilisateur
  - mot de passe
  - email
  - des données de votre choix pour faire les plots (bar, plot, histogramme, camembert etc.. explorez seaborn!).
- Hasher le mot de passe avant de le stocker dans le DataFrame.
- Ajouter le nouvel utilisateur au DataFrame.


In [95]:
def creation_user():
    users_df = pd.read_csv(USERS_FILE)
    while True:
        try:
            nom = str(input("Entrez le nom d'utilisateur"))
            
            user_row = users_df[users_df["username"] == nom]

            if not user_row.empty:
                print("L'utilisateur existe déjà")
                continue
            
            print("Saisiez les information")
            password = hash_password(str(input("Entrez le mot de pass")))
            email = str(input("Entrez une address email"))
            donne_supl = str(input("Entrez des données supplémentaires"))

            users_df.loc[len(users_df)] = [nom, password, email, donne_supl]
            users_df.to_csv(USERS_FILE, index=False)
            print("L'utilisateur a été créé")
            break

        except Exception as e:
            print(e)
            break


### <span style=color:red>Affichage de la liste des utilisateurs (CRUD - Read) :</span>
- Après la connexion d'un utilisateur (administrateur ou non), afficher la liste des autres utilisateurs présents dans le DataFrame.
- Masquer les mots de passe pour des raisons de sécurité.


In [108]:
def read_user(user_nom=''):
    try:
        users_df = pd.read_csv(USERS_FILE, usecols=["username", "donne_s"])
        if not user_nom:
            print(users_df)
        else:
            print(users_df[users_df["username"] == user_nom])
    except Exception as e:
        print("Erreur read_user: {e}")
    

### <span style=color:red>Modification des informations d'un utilisateur (CRUD - Update) :</span>
- Permettre à l'administrateur de modifier les informations d'un utilisateur existant (nom d'utilisateur, mot de passe, email).
- Demander à l'administrateur de saisir le nom d'utilisateur de l'utilisateur à modifier.
- Si l'utilisateur existe, demandez les nouvelles informations et mettez à jour le DataFrame.
- Hasher le nouveau mot de passe si nécessaire.


In [97]:
def update_user(user_nom=''):
    try:
        users_df = pd.read_csv(USERS_FILE)

        def renouveler(mask, users_df:pd.DataFrame):
            # demande des nouvelles information 
            password = hash_password(str(input("Entrez un nouveau mot de pass")))
            email = str(input("Entrez une nouvelle adresse e-mail"))
            donne_supl = str(input("Entrez des données supplémentaires"))

            # Renouvele
            users_df.loc[mask, "password_hash"] = password
            users_df.loc[mask, "email"] = email
            users_df.loc[mask, "donne_s"] = donne_supl

            # Stocke
            users_df.to_csv(USERS_FILE, index=False)
            print("Les informations ont été renouvelées.")

        if not user_nom:
            # Verifie
            nom = str(input("Entrez le nom d'utilisateur à modifier"))
            mask = users_df["username"] == nom

            user_row = users_df[mask]

            if user_row.empty:
                print("L'utilisateur n'existe pas")
            else:
                renouveler(mask, users_df)
        else:
            mask = users_df["username"] == user_nom
            renouveler(mask, users_df)
         
    except Exception as e:
        print(f"{e}")

### <span style=color:red>Statistic</span>

In [98]:
def stat_users():
    try:
        data_users = pd.read_csv(USERS_FILE, usecols=["username", "donne_s"])
        print(data_users.describe())
    except Exception as e:
        print(f"{e}")

### <span style=color:red>Suppression d'un utilisateur (CRUD - Delete) :</span>
- Permettre à l'administrateur de supprimer un utilisateur existant.
- Demander à l'administrateur de saisir le nom d'utilisateur de l'utilisateur à supprimer.
- Si l'utilisateur existe, le supprimer du DataFrame.


In [99]:
def delete_user(user_nom=""):
    try:
        users_df = pd.read_csv(USERS_FILE)

        def supprimer(mask, users_df):
            # index des lignes à supprimer
            idx = users_df.index[mask]

            # supprimer les lignes
            users_df.drop(index=idx, inplace=True)

            # sauvegarder
            users_df.to_csv(USERS_FILE, index=False)
            print("L'utilisateur a été supprimé.")

        if not user_nom:
            # Verifie
            nom = str(input("Entrez le nom d'utilisateur à suprimer"))
            mask = users_df["username"] == nom

            user_row = users_df[mask]

            if user_row.empty:
                print("L'utilisateur n'existe pas")
            else:
                # Demander confirmation
                confirmation = input("Êtes-vous sûr ? (Y/N) : ").upper()
                if confirmation == "Y":
                    supprimer(mask, users_df)
                else:
                    print("Opération annulée.")
        else:
            # Demander confirmation
            confirmation = input("Êtes-vous sûr ? (Y/N) : ").upper()
            
            if confirmation == "Y":
                mask = users_df["username"] == user_nom
                supprimer(mask, users_df)
            else:
                print("Opération annulée.")

    except Exception as e:
        print(f"{e}")

# <span style=color:red>Menu admin</span>


In [100]:
def menu_admin():
    """
    La fonction affiche un menu interactif destiné à l’administrateur.
    Elle permet d’exécuter différentes opérations sur les utilisateurs : création, lecture, mise à jour, suppression et consultation de statistiques.
    La fonction utilise match/case pour gérer les choix de manière claire et structurée. 
    Elle inclut également une gestion d’erreurs afin d’éviter l’arrêt du programme en cas de saisie incorrecte ou d’exception.
    """
    try:
        while True:
            clear_output(wait=True)
            print("\n--- Menu Administrateur ---")
            print("Fonctions possibles :")
            print("  c : créer un utilisateur")
            print("  r : lire les informations")
            print("  u : mettre à jour un utilisateur")
            print("  d : supprimer un utilisateur")
            print("  s : statistiques")

            action = input("Choisissez une action (c/r/u/d/s) : ").strip().lower()
            clear_output(wait=True)
            match action:
                case "c":
                    creation_user()
                case "r":
                    read_user()
                case "u":
                    update_user()
                case "d":
                    delete_user()
                case "s":
                    stat_users()
                case _:
                    print("Erreur de saisie.")
                    continue

            # Continuer ou quitter
            choix = input("Continuer ? (c = continuer, e = exit) : ").strip().lower()

            match choix:
                case "c":
                    continue
                case "e":
                    break
                case _:
                    print("Erreur de saisie. Retour au menu.")

    except Exception as e:
        print(f"Erreur : {e}")


# <span style=color:red>Menu user</span>

In [101]:
def menu_user(nom):
    """
    La fonction affiche un menu interactif destiné à l’utilisateur.
    Elle permet d’exécuter différentes opérations sur les utilisateurs : lecture, mise à jour, suppression et consultation de statistiques.
    La fonction utilise match/case pour gérer les choix de manière claire et structurée. 
    Elle inclut également une gestion d’erreurs afin d’éviter l’arrêt du programme en cas de saisie incorrecte ou d’exception.
    """
    try:
        while True:
            clear_output(wait=True)
            print("\n--- Menu Utilisateur ---")
            print("Fonctions possibles :")
            print("  r : lire les informations")
            print("  u : mettre à jour votre donnes")
            print("  d : supprimer votre donnes")
            print("  s : statistiques")

            action = input("Choisissez une action (r/u/d/s) : ").strip().lower()
            clear_output(wait=True)
            match action:
                case "r":
                    read_user(user_nom=nom)
                case "u":
                    update_user(user_nom=nom)
                case "d":
                    delete_user(user_nom=nom)
                case "s":
                    stat_users()
                case _:
                    print("Erreur de saisie.")
                    continue

            # Continuer ou quitter
            choix = input("Continuer ? (c = continuer, e = exit) : ").strip().lower()

            match choix:
                case "c":
                    continue
                case "e":
                    break
                case _:
                    print("Erreur de saisie. Retour au menu.")
                    

    except Exception as e:
        print(f"Erreur user_menu: {e}")


### Le boucle d'interaction

In [102]:
def interaction():
    try:
        print("Choisissez le connection: admin/a ou utilisateur/u ")
        connect = str(input()).lower()
        
        match connect:
                case "a":
                    connection_admin()
                case "u":
                    connection_user()
                case _:
                    print("Erreur de saisie.")
    except Exception as e:
        print(e)

In [109]:
interaction()


Empty DataFrame
Columns: [username, donne_s]
Index: []
